# Problem Set 2 - Decision Trees, Ensemble Methods
***
**Name**:Jahoon Koo
***

This assignment is due on Canvas by **11:59PM on Wednesday February 22**.

Submit only this Jupyter notebook to Canvas with the name format `PS2_<yourname>.ipynb`. Do not compress it using tar, rar, zip, etc.
Your solutions to analysis questions should be done in Markdown directly below the associated question.

Remember that you are encouraged to discuss the problems with your classmates and instructors, 
but **you must write all code and solutions on your own**, and list any people or sources consulted.
The only exception to this rule is that you may copy code directly from your own solution to homework 1.
***

## Overview 

Your task for this homework is to build a decision tree classifier from scratch. Of course, we provide some initial classes
that you'll be editing. Since last two problems will use the scikit-learn's DecisionTreeClassifier, your solution
does not have to be efficient as long as it passes the sanity checks in a reasonable time (typically less than ~1min).

We will run a small comparison between our implementation and Scikit's in Problem 2 to make sure we didn't miss anything.

The third part will introduce k-fold cross validation to find out how deep is the best decision tree classifier. The last problem
requires a _weak learner_ (implemented as `base` model), so we'll use a decision tree that yields lower performance. But with _Ensemble Methods_,
we will be able to improve the performance by aggregating predictions from multiple weak learners.
For the ensemble methods, we'll explore bagging, Random Forest, and boosting (AdaBoost).

Any Machine Learning interview will almost certainly have a question or two about decision trees and how they're trained.
So understanding the code and trying to implement everything on your own will be the best way to prepare for such interviews.

Also remember, if your code is correct then the sanity checks should pass without any major issue.
But if the sanity checks pass that does not necessarily imply your code is 100% correct.

Happy coding!

In [1]:
import numpy as np
import matplotlib.pylab as plt
import tests
import data
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

### Problem 1 - Decision Trees [30 points]
***
The goal of this problem is to implement the core elements of the Decision Tree classifier.
We do not expect a highly efficient implementation of the functions since the ensemble methods will use the implementation from scikit-learn.


|Age|Salary|Colorado Resident| Has Siblings | College degree|
|:------:|:-----------:| :----------:| :----------:|--:|
| 37 | 44,000 | Yes | No  | Yes|
| 61 | 52,000 | Yes | No  | No |
| 23 | 44,000 | No  | No  | Yes|
| 39 | 38,000 | No  | Yes | Yes|
| 48 | 49,000 | No  | No  | Yes|
| 57 | 92,000 | No  | Yes | No |
| 38 | 41,000 | No  | Yes | Yes|
| 27 | 35,000 | Yes | No  | No |
| 23 | 26,000 | Yes | No  | No |
| 38 | 45,000 | No  | No  | No |
| 32 | 50,000 | No  | No  | Yes|
| 25 | 52,000 | Yes | No  | Yes|

In [2]:
features = np.array([
    [37, 44000, 1, 0],
    [61, 52000, 1, 0],
    [23, 44000, 0, 0],
    [39, 38000, 0, 1],
    [48, 49000, 0, 0],
    [57, 92000, 0, 1],
    [38, 41000, 0, 1],
    [27, 35000, 1, 0],
    [23, 26000, 1, 0],
    [38, 45000, 0, 0],
    [32, 50000, 0, 0],
    [25, 52000, 1, 0]
])
labels = np.array([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1])

Each leaf node (terminal node) in a decision tree has a label value assigned to it. The same label will be assigned
to all samples that reach the leaf node.
- 1.1 [2 pts] What is the best accuracy for a baseline classifier that predicts one label for all rows on the dataset above?
which label should it predict?

% Write-up for 1.1 <br>
#BEGIN <br>
The best accuracy for a baseline classifier is 58.3% when it predicts all rows on the dataset above 1 because there are seven 1s out of 12 labels.

#END<br>

- 1.2 [3 pts] Complete `compute_label` to return the label that should be assigned to the leaf node based on training labels in `y`.

If more than one label are possible, choose the one with the lowest value (e.g, if both `0` and `1` are possible,
choose `0`)

In [3]:
class Node:
    """Base class for LeafNode and ParentNode"""
    left_child = None
    right_child = None
    def feature_importance(self, importance_dict):
        return importance_dict

class LeafNode(Node):
    def __init__(self, y):
        """
        :param y: 1-d array containing labels, of shape (num_points,)
        """
        self.label = self.compute_label(y)

    @staticmethod
    def compute_label(y):
        """
        return the label that yields best performance if predicted of all instances in y
        :param y:  1-d array containing labels
        :return: single label, integer
        """
        node_label = None
        #Workspace 1.2
        #TODO: Return the label that should be assigned to the leaf node
        #In case of multiple possible labels, choose the one with the lowest value
        #Make no assumptions about the number of class labels
        #BEGIN
        
        unique_labels, label_counts = np.unique(y, return_counts=True)
        #np.argmax returns the index of the lowest value in case of multiple possible labels
        node_label = unique_labels[np.argmax(label_counts)]
        
        #END
        return node_label


    def predict(self, x):
        """
        return the label for one obervation x
        :param x: one sample, of shape (num_features)
        :return: label, integer
        """
        return self.label

In [4]:
# Test cell, uncomment to run the tests
tests.test_leaf(LeafNode)

Question 1.2: [PASS]


The tree also contains _parent nodes_. They can either be parents of: leaf nodes, parent nodes, or a combination of the two.
Each parent node has a left and a right child. A parent node is used when we can reduce the impurity of the labels by splitting
the training instances based on a certain threshold.

First, we'll need to choose an impurity measure. For classification,
there are two mainstream measures: _gini index_ and _entropy_. We'll be using the former for our implementation.

\begin{align}
\text{Gini}(y) = 1 - \sum_{c}  (p_c)^2 \text{  and  Entropy}(y) = -\sum_{c}  p_c . \log p_c ,
\end{align}

where $p_c$ is the probability of occurrence (ratio)  of class $c$ among the labels in $y$. *Make sure that the log function being used for entropy is np.log() as it calculates $log_e()$*  

- 1.3 [3 pts] Complete the function `gini` that returns the gini index of labels in `y`.

_Hint: Make sure you handle multi-class labels
(not just binary)._

In [5]:
def gini(y):
    """
    :param y: 1-d array contains labels, of shape (num_points,)
    :return: float, entropy measure of the labels
    """
    gini_index = 0
    # Workspace 1.3
    #TODO: Compute the gini index of the labels
    #BEGIN
    
    unique_labels, label_counts = np.unique(y, return_counts=True)
    label_probability = {}
    number_of_labels = len(y)
    for label, count in zip(unique_labels, label_counts):
        label_probability[label] = count / number_of_labels

    for label in label_probability:
        gini_index += (label_probability[label] ** 2)
    
    #END
    gini_index = 1 - gini_index
    return gini_index

In [6]:
# Test cell, uncomment to run the tests
tests.test_gini(gini)

Question 1.3: [PASS]


Now that we're at a parent node, we decide to partition our label instances in $S$ to two parts indexed by $P_1$ and $P_2$,
and we want to compute how much this split reduces the impurity.

Using the impurity measure $\mathcal{M}$, this impurity reduction is computed as follows:
\begin{align}
\text{Reduction}(S, {P_1, P_2}) = \mathcal{M}(S) - \big[
    \frac{|P_1|}{|S|} .\mathcal{M}(S[P_1]) + \frac{|P_2|}{|S|}.\mathcal{M}(S[P_2])
    \big],
\end{align}

where $|A|$ denotes the size of the set $A$.

The main questions will be based on the entropy measure, in which case the `Reduction` is also called _information gain_
(reducing the entropy implies that the partitioning decision variable and the labels have a higher mutual information).

-  1.4 [3 pts] Complete the `impurity_reduction` function to return the impurity reduction of the split using the provided measure.

In [7]:
def impurity_reduction(y, left_indices, right_indices, impurity_measure=gini):
    """
    :param y: all labels
    :param left_indices: the indices of the elements of y that belong to the left child
    :param right_indices: the indices of the elements of y that belong to the right child
    :param impurity_measure: function that takes 1d-array of labels and returns the impurity measure, defaults to gini
    :return: impurity reduction of the split
    """
    impurity_reduce = 0
    # Workspace 1.4
    #BEGIN
    impurity_y = impurity_measure(y)
    impurty_left = impurity_measure([y[index] for index in left_indices])
    impurty_right = impurity_measure([y[index] for index in right_indices])
    
    impurity_reduce = impurity_y - (((len(left_indices)/len(y)) * impurty_left) + ((len(right_indices)/len(y)) * impurty_right))
    #END
    return impurity_reduce

In [8]:
# Test cell, uncomment to run the tests
tests.test_information_gain(impurity_reduction, gini)

Question 1.4: [PASS]


We'll use `best_partition` to look up for the feature and threshold that yields the partition with the best impurity reduction.

For each feature:
 - Compute all possible thresholds (use `split_values`)
 - For each threshold:
    - Split to `(left_indices, right_indices)` based on the threshold
    - Compute the impurity reduction of the split

The function then returns the feature and the threshold that yield the best impurity reduction (and the reduction value)

 - 1.5 [5 pts] Complete `best_partition`.
 
 _Hint: `split_values` is provided as a helper function. It takes the feature column and returns
the set of thresholds_

In [9]:
def split_values(feature_values):
    """
    Helper function to return the split values. if feature consists of the values f1 < f2 < f3 then
    this returns [(f2 + f1)/2, (f3 + f2)/2]
    :param feature_values: 1-d array of shape (num_points)
    :return: array of shape (max(m-1, 1),) where m is the number of unique values in feature_values
    """
    unique_values = np.unique(feature_values)
    if unique_values.shape[0] == 1:
        return unique_values
    return (unique_values[1:] + unique_values[:-1]) / 2


def best_partition(X, y, impurity_measure=gini):
    """
    :param X: features array, shape (num_samples, num_features)
    :param y: labels of instances in X, shape (num_samples)
    :param impurity_measure: function that takes 1d-array of labels and returns the impurity measure
    :return: Return the best value and its corresponding threshold by splitting based on the different features.
    """

    best_feature, best_threshold, best_reduction = 0, 0, -np.inf

    #Workspace 1.5
    #TODO: Complete the function as detailed in the question and return description
    #BEGIN
    n_samples, n_features = X.shape
    
    for feature in range(n_features):
        feature_values = X[:, feature]
        thresholds = split_values(feature_values)
        for threshold in thresholds:
            left_indices = np.flatnonzero(X[:, feature] < threshold)
            right_indicies = np.flatnonzero(X[:, feature] >= threshold)
            reduction = impurity_reduction(y, left_indices, right_indicies, impurity_measure)
            if reduction > best_reduction:
                best_feature = feature
                best_threshold = threshold
                best_reduction = reduction
    #END
    return best_feature, best_threshold, best_reduction

In [10]:
# Test cell, uncomment to run the tests
# If you chose to not use split_values, then this test will likely fail
tests.test_best_partition(best_partition, gini)

Question 1.5: [PASS]


We provide the implementation of the parent node below. Note that the `left_child` will take instance for which
`feature_id` value is < `feature_threshold`. We should construct our decision tree as such.

In [11]:
class ParentNode:

    def __init__(self, feature_id, feature_threshold, left_child: Node, right_child: Node, weighted_impurity=0):
        """
        Initialize a parent node.
        :param feature_id: the feature index on which the splitting will be done
        :param feature_threshold: the feature threshold. Left child takes item with features[features_id] < threshold
        :param left_child: left child node
        :param right_child: right child node
        :param weighted_impurity: weighted impurity reduction, optional (used for the bonus question)
        """
        self.feature_id = feature_id
        self.threshold = feature_threshold
        self.left_child = left_child
        self.right_child = right_child
        self.weighted_impurity = weighted_impurity

    def feature_importance(self, importance_dict):
        """
        :param importance_dict: dictionary, keys are features indices and value are feature importances
        :return: updated feature importrances dictionary
        """
        #Workspace 2.5.a (bonus)
        #BEGIN
        
        #END
        return importance_dict

    def predict(self, x):
        """
        Predict the label of row x. If we're a leaf node, return the value of the leaf. Otherwise, call predict
        of the left/right child (depending on x[feature_index).
        This will be called by DecisionTree.predict
        :param x: 1-d array of shape (num_features)
        :return: integer, the label for x
        """
        if x[self.feature_id] < self.threshold:
            label = self.left_child.predict(x)
        else:
            label = self.right_child.predict(x)
        return label

Now we tackle the core of a decision tree. The tree is built in a recursive way. The recursion in `DecisionTree.build` works as follows:
- Parameters: `min_samples_split`, `impurity_measure`
- Inputs: `features`, `labels`, `depth`
- Base case of the recursion, return a leaf node if either:
    - `depth` is 0
    - `labels` contains less than `min_samples_split` elements
    - There is no impurity reduction (reduction<=0 for all splits)
- Recursion (there is a split with impurity reduction > 0):
    - create the left and right child nodes with `depth - 1`
    - return the parent node

The left child node will contain instances for which the feature with index `best_feature` is strictly lower than
`best_threshold` of the partition. The right child takes the remaining instances.

- 1.6 [6 pts] Complete `build` method of `DecisionTree`
- 1.7 [2 pts] Complete the `score` method that returns the accuracy on the given data

In [12]:
class DecisionTree:

    def __init__(self, max_depth=-1, min_samples_split=2, impurity_measure=gini):
        """
        Initialize the decision tree
        :param max_depth: maximum depth of the tree
        :param min_samples_split: minimum number of samples required for a split
        :param impurity_measure: impurity measure function to use for best_partition, default to entropy
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_measure = impurity_measure
        self.root = None
        self.num_features = None

    def build(self, X, y, depth) -> Node:
        """
        Recursive method used to build the decision tree nodes
        :param X: data that are used to build the tree, of shape (num_samples, num_features)
        :param y: labels of the samples in features, of shape (num_samples)
        :param depth: depth of the tree to create
        :return: the root node of the tree
        """
        # Workspace 1.6
        #BEGIN
        n_samples, n_features = X.shape 
        
        feature_id, feature_threshold, best_reduction = best_partition(X, y, self.impurity_measure)    
        if depth == 0 or n_samples < self.min_samples_split or best_reduction <= 0:
            return LeafNode(y)
        else:
            
            left_indices = np.flatnonzero(X[:, feature_id] < feature_threshold)
            right_indicies = np.flatnonzero(X[:, feature_id] >= feature_threshold)

            left_child = self.build(X[left_indices], y[left_indices], depth - 1)
            right_child = self.build(X[right_indicies], y[right_indicies], depth - 1)
            
            return ParentNode(feature_id, feature_threshold, left_child, right_child)
                        
        
        #END

    def fit(self, X, y):
        """
        :param X: Training samples
        :param y: training labels
        :return: trained classifier
        """
        self.num_features = X.shape[1]
        self.root = self.build(X, y, self.max_depth)
        return self

    def compute_importance(self, features_names=None):
        """
        Compute the normalized feature importances
        :param features_names: Name of features to use, defaults to integers
        :return: Dictionary with feature_name: feature_importance
        """
        if features_names is None:
            features_names = ["feat_%i" % i for i in range(self.num_features)]
        feats_importances = {i:0.0 for i in range(self.num_features)} # to include
        # Workspace 2.5.b (bonus)
        # ToDo: Call the root's feature and importance and scale values in feats_importance to sum to 1
        total_importances = 1
        #BEGIN
        
        #END
        return {features_names[k] :v for k,v in feats_importances.items() if v>0}

    def predict(self, X):
        """
        Loops through rows of X and predicts the labels one row at a time
        """
        y_hat = np.zeros((X.shape[0],), int)
        for i in range(X.shape[0]):
            y_hat[i] = self.root.predict(X[i])
        return y_hat

    def score(self, X, y):
        """
        Return the mean accuracy on the given test data and labels.
        :param X: Test samples, shape (num_points, num_features)
        :param y: true labels for X, shape (num_points,)
        :return: mean accuracy
        """
        accuracy = 0
        # Workspace 1.7
        #BEGIN
        y_hat = self.predict(X)
        for i in range(len(y)):
            if y_hat[i] == y[i]:
                accuracy += 1
        
        accuracy /= len(y)
        #END
        return accuracy

In [13]:
# Test cell, uncomment to run the tests
# If you chose to not use split_values, then this test will likely fail
tests.test_tree_build(DecisionTree, gini)

Question 1.6: [PASS]


- 1.8 [2 pts] We want to evaluate our `DecisionTree(max_depth=3, min_samples_split=2`.
What's the accuracy we achieve on the training data using the tree? ( we train and evaluate using `(features, labels)`)

In [14]:
# Workspace 1.8
#BEGIN
decision_tree = DecisionTree(max_depth=3, min_samples_split=2).fit(features, labels)
accuracy = decision_tree.score(features, labels)
print(accuracy)
#END

0.9166666666666666


- 1.9 [2 pts] Using `min_samples_split=2`, what is the minimum depth so that our `DecisionTree` fits perfectly our
training data `(labels, features)`.

In [15]:
# Workspace 1.9
# To show that the minimum required depth is n, you can provide the accuracy for depth = (n-1) and depth = n
#BEGIN
min_depth = -1
for depth in range(1,10):
    decision_tree = DecisionTree(max_depth=depth, min_samples_split=2).fit(features, labels)
    accuracy = decision_tree.score(features, labels)
    print("depth: {}, accuracy: {}".format(str(depth), str(accuracy)))
    if accuracy == 1:
        min_depth = depth
        break

print("minimum depth is {}.".format(str(min_depth)))
#END

depth: 1, accuracy: 0.75
depth: 2, accuracy: 0.9166666666666666
depth: 3, accuracy: 0.9166666666666666
depth: 4, accuracy: 0.9166666666666666
depth: 5, accuracy: 1.0
minimum depth is 5.


We provide an example below to display the structure of a decision tree. Look at print_tree() in tests.\_\_init\_\_.py to understand how this visualization is working.
- 1.10 (2pts) Edit it to show the tree for the required minimum depth found in 1.8

In [16]:
tree = DecisionTree(max_depth=5, min_samples_split=2).fit(features, labels)
tests.print_tree(tree, ["age", "salary", "resident", "siblings"])

                  ┌│label: 0
       ┌|salary  │┘
       │|36500.00│┐
       │          │       ┌│label: 1
       │          └|age  │┘
       │           |37.50│┐
       │                  │                  ┌│label: 1
       │                  │       ┌|salary  │┘
       │                  │       │|43000.00│┐
       │                  │       │          └│label: 0
       │                  └|age  │┘
       │                   |38.50│┐
       │                          └│label: 1
|age  │┘
|52.50│┐
       └│label: 0


### Problem 2: DecisionTree vs DecisionTreeClassifier [6 points]

We've just showed that our decision tree is better than the naive NaiveBayes! Let see how it compares to scikit's
DecisionTreeClassifier.

First, we'll need a fancier dataset. We are going to predict the level of usage of a bike sharing system in Washington, DC using the decision trees.

We start by loading preprocessed data that we'll use. Since the original Bike Sharing
 [dataset](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset)
 is for regression, we have to transform `BikeSharing.y_train` and `BikeSharing.y_test` to discrete values reflecting the level of usage.
We have included this dataset with the homework -- you can find it in the data directory.

|Bike Rentals| Label|
|:----------:|--:|
| $ P < $2000|0|
|2000$\leq P < $ 4000| 1 |
|4000$ \leq P < $ 6000| 2 |
|6000$ \leq P $ | 3 |

- 2.1 [3 pts] Start by transforming `y_train` and `y_test` of `bike_sharing` to discrete values using the provided ranges

In [17]:
bike_sharing = data.BikeSharing()
#Workspace 2.1
#TODO: Discretize y_train and y_test
#BEGIN
def regression_data_to_discrete_data(y):
    for i in range(len(y)):
        if y[i] < 2000:
            y[i] = 0
        elif y[i] >= 2000 and y[i] < 4000:
            y[i] = 1
        elif y[i] >= 4000 and y[i] < 6000:
            y[i] = 2
        elif y[i] >= 6000:
            y[i] = 3

regression_data_to_discrete_data(bike_sharing.y_train)       
regression_data_to_discrete_data(bike_sharing.y_test)         
#END
print(np.unique(bike_sharing.y_train), bike_sharing.X_train.shape)
print(np.unique(bike_sharing.y_test), bike_sharing.X_test.shape)

[0. 1. 2. 3.] (584, 12)
[0. 1. 2. 3.] (147, 12)


- 2.2 [3 pts] Compare our `DecisionTree` and scikit's `DecisionTreeClassifier` on the bike sharing dataset by reporting the accuracies on the test data.

 [scikit's `DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
uses Gini Index by default and shuffles the features before each split. Refer to the documentation for more information about how to change the impurity measure if you are curious.

Use `max_depth = 5, min_samples_split=2, random_state=11` for the comparison.

In [18]:
# Workspace 2.2.a
#BEGIN
from sklearn.tree import DecisionTreeClassifier

our_tree = DecisionTree(max_depth=5, min_samples_split=2).fit(bike_sharing.X_train, bike_sharing.y_train)
our_tree_accuracy = our_tree.score(bike_sharing.X_test, bike_sharing.y_test)
print("The accuracy of our Decision Tree on the test data is {}.".format(str(our_tree_accuracy)))

their_tree = DecisionTreeClassifier(max_depth = 5, min_samples_split=2, random_state=11).fit(bike_sharing.X_train, bike_sharing.y_train)
their_tree_accuracy = their_tree.score(bike_sharing.X_test, bike_sharing.y_test)
print("The accuracy of scikit's DecisionTreeClassifier on the test data is {}.".format(str(their_tree_accuracy)))
#END

The accuracy of our Decision Tree on the test data is 0.7074829931972789.
The accuracy of scikit's DecisionTreeClassifier on the test data is 0.7074829931972789.


% Write-up for 2.2.b <br>
#BEGIN <br>
The accuracy of our Decision Tree on the test data is 0.7074829931972789.
The accuracy of scikit's DecisionTreeClassifier on the test data is 0.7074829931972789.

They both have the same accuracy when max_depth=5, min_samples_split=2, and random_state=11.

#END<br>

### Bonus questions
We've implemented `DecisionTree` to handle different measures of impurity. We want now to compare our implementation
to the standard `DecisionTreeClassifier` using Gini index.
- **(Bonus)** 2.3  [2 pts] Complete `entropy` function
_hint: for the log function, use `np.log` and the convention `0 * log(0) = 0`._

In [19]:
def entropy(y):
    """
    :param y: 1-d array contains labels, of shape (num_points,)
    :return: float, gini index the labels
    """
    entropy_value = 0
    # Workspace 2.3
    #TODO: Compute the gini index of the labels in y
    #BEGIN
    unique_labels, label_counts = np.unique(y, return_counts=True)
    label_probability = {}
    number_of_labels = len(y)
    for label, count in zip(unique_labels, label_counts):
        label_probability[label] = count / number_of_labels

    for label in label_probability:
        if label_probability[label] == 0:
            entropy_value += 0
        else:
            entropy_value += (label_probability[label] * np.log(label_probability[label]))
    entropy_value *= -1
    #END
    return entropy_value

- **(Bonus)** 2.4 [2 pts] Perform the same comparison as in 2.2 with entropy but without setting the random state.
How do you explain the result?

In [20]:
np.random.seed(2) # to fix the randomness in DecisionTreeClassifier
# Workspace 2.4.a
#BEGIN
from sklearn.tree import DecisionTreeClassifier

our_tree = DecisionTree(max_depth=5, min_samples_split=2, impurity_measure=entropy).fit(bike_sharing.X_train, bike_sharing.y_train)
our_tree_accuracy = our_tree.score(bike_sharing.X_test, bike_sharing.y_test)
print("The accuracy of our Decision Tree on the test data is {}.".format(str(our_tree_accuracy)))

their_tree = DecisionTreeClassifier(max_depth = 5, min_samples_split=2, criterion="entropy").fit(bike_sharing.X_train, bike_sharing.y_train)
their_tree_accuracy = their_tree.score(bike_sharing.X_test, bike_sharing.y_test)
print("The accuracy of scikit's DecisionTreeClassifier on the test data is {}.".format(str(their_tree_accuracy)))
#END

The accuracy of our Decision Tree on the test data is 0.7142857142857143.
The accuracy of scikit's DecisionTreeClassifier on the test data is 0.7074829931972789.


% Write-up for 2.4.b <br>
#BEGIN <br>
The accuracy of our Decision Tree on the test data is 0.7142857142857143.
The accuracy of scikit's DecisionTreeClassifier on the test data is 0.7074829931972789.

Our decision tree performs slightly better than scikit's DecisionTreeClassifier when using entropy as impurity measure.

#END<br>

**(Bonus)**

Now we can be a bit more ambitious and compute the importance of each feature in our decision tree. The importance of feature $f$
is the sum of the weighted impurity reduction of parent nodes that are split based on the feature $f$.

The weighted impurity reduction of $node_i$ is the following:

\begin{align}
\frac{N_{\text{node}_i}}{N_\text{total}} \times \text{impurity reduction}({\text{node}_i}),
\end{align}

where $N$ is the total number of training samples, and $N_{\text{node}_i}$ is the number of training samples that at $node_i$.

Since we scale the feature importances in `DecisionTree` to sum to 1, we don't have to divide by $N_\text{total}$
and we can simply use:

\begin{align}
\text{weighted impurity}(\text{node}_i) = N_{\text{node}_i} \times \text{impurity reduction}({\text{node}_i}),
\end{align}

Practically, we use a dictionary `feats_importances` that maps feature indices to their importances.
- Start with `feats_importance[f]=0` for all `f`
- Start the recursion from the root node:
    - Current node is split based on feature `i`
    - add weighted impurity reduction to `feature_importance[i]`
    - ask right and left child to do the same
- Scale the values in `feats_importance` to sum to 1   
- return `feats_importance`

You can provide `weighted_impurity` directly when initializing the parent nodes in `DecisionTree.build`.

- **(Bonus)** 2.5 [4 pts] Complete `ParentNode`'s `feature_importance`, `DecisionTree`'s `compute_importance`, and 
compare our implementation to that of scikits on bike sharing data.

Use `random_state=0, splitter="best"` for scikit and `max_depth=3`, `min_samples_split=2`, gini index for both.
Note that scikit's DecisonTreeClassifier always uses Gini for the feature importance computation (even if `criterion` is set to Entropy).

In [21]:
# Workspace 2.5
# Compare feature importances of DecisionTree(gini) to DecisionTreeClassifier
# Exclude features with 0 importance from both
#BEGIN

#END

### Problem 3 - Model Selection via Cross Validation [16 points]
***
In this problem, we will be working with scikit-learn `DecisionTreeClassifier`. We want to figure out the best `max_depth`
 for our dataset.

In the bike sharing dataset, we only have a training set and a test set. The question then is how do we perform the model
 selection seen in Problem Set 1?

One way to do so is via **the cross validation set approach** which basically means setting aside a portion of
our training data to use as a validation set. The goal is to use the validation set to find the best hyperparameters
for our model (`max_depth` in the case of decision trees).

- 3.1 [3 points] complete the `cross_validate` function to train the classifier on the training set and
return the accuracy on the validation set based on provided indices.

In [22]:
def cross_validate(classifier, X, y, train_indices, valid_indices):
    """
    Train classifier on training set and validate on the validation set
    :param classifier: the classifier to use
    :param X: all data of shape (num_samples, num_features)
    :param y: all labels of shape (num_samples)
    :param train_indices:  indices to be used for training the model
    :param valid_indices:  indices to be used for validating the model
    :return: he accuracy of the classifier on the validation set
    """
    valid_accuracy = 0
    #Workspace 3.1
    #TODO: train and validate the model based on provided indices
    #Hint: use score method of the classifier
    #BEGIN 

    classifier.fit(np.array([X[index] for index in train_indices]), np.array([y[index] for index in train_indices]))
    valid_accuracy = classifier.score(np.array([X[index] for index in valid_indices]), np.array([y[index] for index in valid_indices]))
    
    #END
    return valid_accuracy

- 3.2 [2 points] Report the validation accuracy using the validation set approach for scikit-learn `DecisionTreeClassifier` with `max_depth=3`
 when using the last 100 training points as a validation set and the rest as training set.

In [23]:
#Workspace 3.2
#TODO: Report the cross validation accuracy using the last 100 training points as validation set
#and the rest of the training points as training
#BEGIN 
from sklearn.tree import DecisionTreeClassifier

n_samples, n_features = bike_sharing.X_train.shape
classifier = DecisionTreeClassifier(max_depth=3, min_samples_split=2)
train_indices = [index for index in range(0, n_samples - 100)]
valid_indices = [index for index in range(n_samples - 100, n_samples)]

valid_accuracy = cross_validate(classifier, bike_sharing.X_train, bike_sharing.y_train, train_indices, valid_indices)

print(valid_accuracy)
#END

0.76


The issue with the validation set approach is that we're reducing the size of our training data,
 and the lower number of samples implies higher uncertainty.

A work-around is to use *k-fold cross validation*.
We start by partitioning the training data into k different and equally size partitions.
Then for each of the k runs, we keep a different chunk for the validation while using the remaining k-1 for training.
We note the validation accuracy during each of the k runs.

After each of the k-folds has been used as a validation set, the average of the k recorded accuracies becomes the performance of our model.
The k-fold cross validation method gives us a better estimate on how well the model would perform on new unseen data
 (test set) while allowing it to train on a larger portion of the dataset.
- 3.3 [5 points] Complete `k_fold_cv`. Use the helper function `generate_folds` that generates the partition of indices to k different chunks.

In [24]:
def generate_folds(size, k):
    """
    Shuffles and partition range(size) to to k contiguous chunks then generates the train/valid indices for the k-fold
    To use as a generator, for an example run:
        for train_idx, valid_idx in generate_folds(10,3): print(train_idx, valid_idx)
    :param size: size of the range that should be split
    :param k: number of folds
    :return: iterable of different k splits, each is a tuple (train_indices, valid_indices)
             where len(valid_indices)~ size/k
    """
    permutation = np.random.RandomState(seed=42).permutation(size)
    split_sizes = [size//k + (i < (size % k)) for i in range(k)] # we split the remainder amongst the first folds
    start = 0
    for i in range(k):
        # valid indices of i-th split for which start <= σ < start + size_split[i]
        # for_valid is True in position where condition is true, False otherwise
        for_valid = np.logical_and(start<= permutation, permutation< start + split_sizes[i])
        start += split_sizes[i] # update the start of the fold
        valid_indices = np.where(for_valid)[0]
        # train indices of i-th split for which σ <start or  start + size_split[i] <= σ
        # ~bool_array is negation of bool_array
        train_indices = np.where(~for_valid)[0]
        yield train_indices, valid_indices

def k_fold_cv(classifier, k, X, y):
    """
    This function performs k-fold cross validation
    :param classifier: a classifier to be used
    :param k: number of folds
    :param X: all training data of shape (num_samples, num_features)
    :param y: all labels of shape (num_samples)
    :return: the average accuracy of the classifier in k-runs
    """
    mean_accuracy = 0
    #Workspace 3.3
    #BEGIN
    n_samples, n_features = X.shape
    
    for train_indices, valid_indices in generate_folds(n_samples, k):
        mean_accuracy += cross_validate(classifier, X, y, train_indices, valid_indices)
    
    mean_accuracy /= k
    #END
    return mean_accuracy

- 3.4 [4 points] Consider depths from 1 to 10. Perform hyperparameter search by doing 8-fold cross validation for each depth. What is the best value of `max_depth` and what is the best cross validation accuracy you find over the validation splits?

In [25]:
np.random.seed(4)  # changing the seed might yield different results
best_depth, best_accuracy = -1, 0

#Workspace 3.4
#TODO: 
#BEGIN 

k = 8

for depth in range(1, 11):
    classifier = DecisionTreeClassifier(max_depth=depth, min_samples_split=2)
    accuracy = k_fold_cv(classifier, k, bike_sharing.X_train, bike_sharing.y_train)
    print("current depth: {}, current accuracy: {}".format(depth, accuracy))
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
    
#END
print("Cross validation accuracy for chosen best max_depth %d: %f" % (best_depth, best_accuracy))

current depth: 1, current accuracy: 0.5017123287671234
current depth: 2, current accuracy: 0.6849315068493151
current depth: 3, current accuracy: 0.7602739726027397
current depth: 4, current accuracy: 0.7226027397260273
current depth: 5, current accuracy: 0.7311643835616438
current depth: 6, current accuracy: 0.75
current depth: 7, current accuracy: 0.7568493150684932
current depth: 8, current accuracy: 0.7534246575342466
current depth: 9, current accuracy: 0.7517123287671234
current depth: 10, current accuracy: 0.7226027397260274
Cross validation accuracy for chosen best max_depth 3: 0.760274


- 3.5 [2 pts] Train a new model on the entire training set with the best `max_depth` you found above. Report the accuracy of the new model.

In [26]:
test_accuracy = 0
#Workspace 3.5
#BEGIN 
test_accuracy = DecisionTreeClassifier(max_depth=best_depth, min_samples_split=2).\
    fit(bike_sharing.X_train, bike_sharing.y_train).\
    score(bike_sharing.X_test, bike_sharing.y_test)

#END
print ("accuracy of the best model on the testing set", test_accuracy)

accuracy of the best model on the testing set 0.6802721088435374


Problem 4  - Decision Tree Ensembles: Bagging, Random Forests and Boosting [48 points]
---

### Training Data for Decision tree Ensembles
***
Please do not change this class

In [27]:
# import numpy as np
from sklearn.base import clone
from collections import Counter
import itertools

In [28]:
class ThreesAndEights:
    """
    Class to store MNIST data
    """

    def __init__(self, location):

        import pickle, gzip

        # Load the dataset
        f = gzip.open(location, 'rb')

        # Split the data set 
#         X_train, y_train, X_valid, y_valid = pickle.load(f)
        train_set, valid_set, test_set = pickle.load(f)
    
        X_train, y_train = train_set
        X_valid, y_valid = valid_set

        # Extract only 3's and 8's for training set 
        self.X_train = X_train[np.logical_or( y_train==3, y_train == 8), :]
        self.y_train = y_train[np.logical_or( y_train==3, y_train == 8)]
        self.y_train = np.array([1 if y == 8 else -1 for y in self.y_train])
        
        # Shuffle the training data 
        shuff = np.arange(self.X_train.shape[0])
        np.random.shuffle(shuff)
        self.X_train = self.X_train[shuff,:]
        self.y_train = self.y_train[shuff]

        # Extract only 3's and 8's for validation set 
        self.X_valid = X_valid[np.logical_or( y_valid==3, y_valid == 8), :]
        self.y_valid = y_valid[np.logical_or( y_valid==3, y_valid == 8)]
        self.y_valid = np.array([1 if y == 8 else -1 for y in self.y_valid])
        
        f.close()

In [29]:
data = ThreesAndEights("data/mnist.pklz")

Feel free to explore this data and get comfortable with it before proceeding further.

### Bagging
Bootstrap aggregating, also called bagging, is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method. Bagging is a special case of the model averaging approach.

Given a standard training set $D$ of size n, bagging generates $N$ new training sets $D_i$, roughly each of size n * ratio, by sampling from $D$ uniformly and with replacement. By sampling with replacement, some observations may be repeated in each $D_i$ The $N$ models are fitted using the above $N$ bootstraped samples and combined by averaging the output (for regression) or voting (for classification). 

-Source [Wiki](https://en.wikipedia.org/wiki/Bootstrap_aggregating)

### Problems 4.1-4.4: Implementing Bagging [19-points]
***

We've given you a skeleton of the class `BaggingClassifier` below which will train a classifier based on the decision trees as implemented by sklearn. Your tasks are as follows, please approach step by step to understand the code flow:
* 4.1 [5 points] Implement `bootstrap` method which takes in two parameters (`X_train, y_train`) and returns a bootstrapped training set ($D_i$)
* 4.2 [5 points] Implement `fit` method which takes in two parameters (`X_train, y_train`) and trains `N` number of base models on different bootstrap samples. You should call `bootstrap` method to get bootstrapped training data for each of your base model
* 4.3 [4 points] Implement `voting` method which takes the predictions from learner trained on bootstrapped data points `y_hats` and returns final prediction as per majority rule. In case of ties, return either of the class randomly.
* 4.4 [5 points] Implement `predict` method which takes in multiple data points and returns final prediction for each one of those. Please use the `voting` method to reach consensus on final prediction.

In [30]:
from sklearn.tree import DecisionTreeClassifier
import random

class BaggingClassifier:
    def __init__(self, ratio = 0.73, N = 20, base=DecisionTreeClassifier(max_depth=4)):
        """
        Create a new BaggingClassifier
        
        Args:
            base (BaseEstimator, optional): Sklearn implementation of decision tree
            ratio: ratio of number of data points in subsampled data to the actual training data
            N: number of base estimator in the ensemble
        
        Attributes:
            base (estimator): Sklearn implementation of decision tree
            N: Number of decision trees
            learners: List of models trained on bootstrapped data sample
        """
        
        assert ratio <= 1.0, "Cannot have ratio greater than one"
        self.base = base
        self.ratio = ratio
        self.N = N
        self.learners = []
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        #TODO: Implement functionality to fit models on the bootstrapped samples
        # cloning sklearn models:
        # from sklearn.base import clone        
        #BEGIN        
        for i in range(self.N):
            h = clone(self.base)
            X_sample, y_sample = self.bootstrap(X_train, y_train)
            learner = h.fit(X_sample, y_sample)
            self.learners.append(learner)
        #END
        
    def bootstrap(self, X_train, y_train):
        """
        Args:
            n (int): total size of the training data
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
        
        
        
        Returns:
            X_train (ndarray): [ratio*n_samples x n_features] ndarray of training data
            y_train (ndarray): [ratio*n_samples] ndarray of training labels
        """
        #TODO: Implement functionality to sample (with replacement) n * ratio
        # number of data points from input examples
        #BEGIN
        n_samples, n_features = X_train.shape
        sample_indicies = np.random.choice(n_samples, int(n_samples * self.ratio))
        
        X_sample = np.array([X_train[index] for index in sample_indicies])
        y_sample = np.array([y_train[index] for index in sample_indicies])
        return X_sample, y_sample
        #END
    
    def predict(self, X):
        """
        BaggingClassifier prediction for data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        
        #TODO: Using the individual classifiers trained predict the final prediction using voting mechanism
        #BEGIN
        n_samples, n_features = X.shape
        y_hats = np.zeros((n_samples, self.N), dtype=int)
        y_hat = np.zeros((n_samples), dtype=int)
        
        for i in range(self.N):
            y_hats[:, i] = self.learners[i].predict(X)
         
        for i in range(n_samples):
            y_hat[i] = self.voting(y_hats[i])
        
        return y_hat
        #END
        
    def voting(self, y_hats):
        """
        Args:
            y_hats (ndarray): [N] ndarray of data
        Returns:
            y_final : int, final prediction of the sample
        """
        #TODO: Implement majority voting scheme and incase of ties return random label
        #BEGIN
        unique_labels, label_counts = np.unique(y_hats, return_counts=True)
        
        max_label_counts = max(label_counts) 
        indicies = [i for i in range(len(label_counts)) if label_counts[i] == max_label_counts]
        
        winners = unique_labels[indicies]
        # if there is one winner, random choice will return that value only.
        # otherwise, it will randomly pick a winner incase of ties
        y_final = random.choice(winners)

        return y_final
        #END
        
    def score(self, X, y):
        """
        Return the mean accuracy on the given test data and labels.
        :param X: Test samples, shape (num_points, num_features)
        :param y: true labels for X, shape (num_points,)
        :return: mean accuracy
        """
        accuracy = 0
        # Workspace 1.7
        #BEGIN
        y_hat = self.predict(X)
        for i in range(len(y)):
            if y_hat[i] == y[i]:
                accuracy += 1
        
        accuracy /= len(y)
        #END
        return accuracy

### Problem 4.5 - BaggingClassifier for Handwritten Digit Recognition [5 points]
***

4.5 a [2 points] After you've successfully completed `BaggingClassifier` find the optimal values of `N` and `depth` using k-fold cross validation. You are allowed to use sklearn library to split your training data in folds. Keep the other hyperparameters unchanged. Use the data from `ThreesAndEights` class initialized variable `data`. 

Hint:  Vary `depth` up to 10, `N` up to 40. The number of decision trees `N` is generally a trade-off between 'improvement in accuracy' vs 'computation time'.

In [158]:
# Just an example, change accordingly
ratios = np.logspace(1, 5, 3, base = .9)
depths = np.linspace(3, 10, 8, dtype = int)
Ns = np.linspace(10, 40, 10, dtype = int)

#Workspace 4.5
#BEGIN
# np.random.seed(4)  # changing the seed might yield different results
best_depth = -1
best_n = -1
best_accuracy = 0

k = 8

for depth in depths:
    for N in Ns:
        classifier = BaggingClassifier(ratio = 0.73, N = N, base=DecisionTreeClassifier(max_depth=depth))
        accuracy = k_fold_cv(classifier, k, data.X_train, data.y_train)
        print("current depth: {}, current N: {}, current accuracy: {}".format(depth, N, accuracy))

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_n = N

print("Cross validation accuracy for chosen best_depth %d best_n %d: %f" % (best_depth, best_n, best_accuracy))
#END

current depth: 3, current N: 10, current accuracy: 0.939455637560678
current depth: 3, current N: 13, current accuracy: 0.9450871741656659
current depth: 3, current N: 16, current accuracy: 0.9396566019305534
current depth: 3, current N: 20, current accuracy: 0.9452882195042642
current depth: 3, current N: 23, current accuracy: 0.9426735775090912
current depth: 3, current N: 26, current accuracy: 0.9399582913915351
current depth: 3, current N: 30, current accuracy: 0.9409644087404765
current depth: 3, current N: 33, current accuracy: 0.9407622298397597
current depth: 3, current N: 36, current accuracy: 0.9431764742461164
current depth: 3, current N: 40, current accuracy: 0.9396570067741673
current depth: 4, current N: 10, current accuracy: 0.9566533133049101
current depth: 4, current N: 13, current accuracy: 0.9607764026049906
current depth: 4, current N: 16, current accuracy: 0.9599724641567658
current depth: 4, current N: 20, current accuracy: 0.960876965758651
current depth: 4, curr

4.5 b [2 points] After finding optimal parameters using k-fold cross-validation,report accuracy on the validation data using the optimal parameter values.

In [39]:
#Workspace 4.5 b
#BEGIN
best_n = 40
best_depth = 10
classifier = BaggingClassifier(ratio = 0.73, N = best_n, base=DecisionTreeClassifier(max_depth=best_depth))
classifier.fit(data.X_train, data.y_train)                                           
accuracy = classifier.score(data.X_valid, data.y_valid)
print("Accuracy on the validation data using the optimal parameter values is {}".format(str(accuracy)))
#END

Accuracy on the validation data using the optimal parameter values is 0.9754781755762629


4.5 c [1 point] What is the most deciding hyperparameter? Give us a brief intuition as to why that might be.

% Write-up for 4.5.c <br>
#BEGIN <br>
The most deciding hyperparameter is the number of base estimator in the ensemble. Increasing the number of base estimators can help to improve the BaggingClassifier's robustness by reducing the impact of any individual tree's biases and overfitting problems.

#END<br>

### Problems 4.6-4.8 Implementing Random Forest [12 points]
Random Forest has an additional layer of randomness compared to Bagging: we also sample a random subset of the features (columns). The rest of the implementation should be similar if not exactly the same as Bagging. In addition to keeping track of the estimators (in RandomForest.estimators, we also have to store the features indices that are used by each estimator (in RandomForest.features_indices).

4.6 [4 points] First, complete `bootstrap` to return a random sample of size sample_ratio* len(X_train) of labels and feature_ratio * num_features of features

4.7 [4 points] Complete `fit` by building n_estimators of DecisionTreeClassifier, each trained on random sample of the data. Make sure to keep track of the sampled features for each estimator to use them in the prediction step

4.8 [4 points] Complete `predict` method to return the most likely label by combining different estimators predictions.

In [31]:
class RandomForest(BaggingClassifier):
    def __init__(self, N = 20, samples_ratio = 0.73, features_ratio = 1.0,base=DecisionTreeClassifier(max_depth=4)):
        """
        Create a new BaggingClassifier
        
        Args:
            N: number of base estimator in the ensemble
            samples_ratio: ratio of number of data points in subsampled data to the actual training data
            features_ratio: ratio of number of features to be considered in each sample
            base (BaseEstimator, optional): Sklearn implementation of decision tree
        
        Attributes:
            base (estimator): Sklearn implementation of decision tree
            N: Number of decision trees
            learners: List of models trained on bootstrapped data sample
            samples_ratio: ratio of number of data points in subsampled data to the actual training data
            features_ratio: ratio of number of features to be considered in each sample
            features_indices_array = array to keep track of feature indices sampled for each learner model
        """
        
        assert samples_ratio <= 1.0, "Cannot have ratio greater than one"
        assert features_ratio <= 1.0, "Cannot have ratio greater than one"
        self.base = base
        self.samples_ratio = samples_ratio
        self.N = N
        self.learners = []
        self.features_ratio = features_ratio
        self.features_indices_array = []
    
    def bootstrap(self, X_train, y_train):
        """
        Args:
            n (int): total size of the training data
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        Returns:
            X_train (ndarray): [samples_ratio*n_samples x features_ratio*n_features] ndarray of training data   
            y_train (ndarray): [samples_ratio*n_samples] ndarray of data
            features_indices: indices of the features sampled to be used in other functions
        """
        #TODO: Implement functionality to sample (with replacement) n_samples* samples_ratio number of data points 
        # from input examples and inturn sample n_features* features_ratio number of features from each data point
        #Workspace 4.6
        #BEGIN
        n_samples, n_features = X_train.shape
        sample_indicies = np.random.choice(n_samples, int(n_samples * self.samples_ratio))
        feature_indices = np.random.choice(n_features, int(n_features * self.features_ratio))
        
        X_sample = np.array([X_train[index] for index in sample_indicies])
        y_sample = np.array([y_train[index] for index in sample_indicies])
        return X_sample, y_sample, feature_indices
        
        #END
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        #TODO: Implement functionality to fit models on the bootstrapped samples
        # cloning sklearn models:
        # from sklearn.base import clone
        # h = clone(self.base)
        #Workspace 4.7
        #BEGIN        
        
        for i in range(self.N):
            h = clone(self.base)
            
            X_sample, y_sample, feature_indices = self.bootstrap(X_train, y_train)
            self.features_indices_array.append(feature_indices)
            
            learner = h.fit(X_sample[:, feature_indices], y_sample)
            self.learners.append(learner)
        #END

    
    def predict(self, X):
        """
        BaggingClassifier prediction for data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        #TODO: compute cumulative sum of predict proba from estimators and return the labels with highest likelihood
        #Workspace 4.8
        #BEGIN
        n_samples, n_features = X.shape
        y_hats = np.zeros((n_samples, self.N), dtype=int)
        y_hat = np.zeros((n_samples), dtype=int)
        
        for i in range(self.N):
            y_hats[:, i] = self.learners[i].predict(X[:, self.features_indices_array[i]])
         
        for i in range(n_samples):
            y_hat[i] = self.voting(y_hats[i])
        
        return y_hat
        #END
        
    def voting(self, y_hats):
        """
        Args:
            y_hats (ndarray): [N] ndarray of data
        Returns:
            y_final : int, final prediction of the sample
        """
        #TODO: Implement majority voting scheme and incase of ties return random label
        #BEGIN
        unique_labels, label_counts = np.unique(y_hats, return_counts=True)
        
        max_label_counts = max(label_counts) 
        indicies = [i for i in range(len(label_counts)) if label_counts[i] == max_label_counts]
        
        winners = unique_labels[indicies]
        # if there is one winner, random choice will return that value only.
        # otherwise, it will randomly pick a winner incase of ties
        y_final = random.choice(winners)

        return y_final
        #END
        
    def score(self, X, y):
        """
        Return the mean accuracy on the given test data and labels.
        :param X: Test samples, shape (num_points, num_features)
        :param y: true labels for X, shape (num_points,)
        :return: mean accuracy
        """
        accuracy = 0
        # Workspace 1.7
        #BEGIN
        y_hat = self.predict(X)
        for i in range(len(y)):
            if y_hat[i] == y[i]:
                accuracy += 1
        
        accuracy /= len(y)
        #END
        return accuracy

### Problems 4.9 Experimenting with Random Forest [2 points]
4.9 [2 points] Keep the optimal hyperparameters for `N` and `depth` found in `BaggingClassifier`, please run K-fold cross-validation with a range of `features_ratio` and report an approximate ratio at which the accuracy stops to improve significantly. You may use any kind of tools necessary.

In [51]:
# Just an example, change accordingly
features_ratios = np.logspace(1, 5, 10, base = .5)

#Workspace 4.9
#BEGIN
best_feature_ratio = -1
best_accuracy = 0

best_depth = 10 
best_n = 40
k = 8

for features_ratio in features_ratios:
    classifier = RandomForest(N = best_n, samples_ratio = 0.73, features_ratio = features_ratio, base=DecisionTreeClassifier(max_depth=best_depth))
    accuracy = k_fold_cv(classifier, k, data.X_train, data.y_train)
    print("current feature ratio: {}, current accuracy: {}".format(features_ratio, accuracy))

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_feature_ratio = features_ratio        

print("Cross validation accuracy for chosen best_feature_ratio %f: %f" % (best_feature_ratio, best_accuracy))    
#END

current feature ratio: 0.5, current accuracy: 0.991954299957378
current feature ratio: 0.3674336230688997, current accuracy: 0.992155102389808
current feature ratio: 0.2700149347230765, current accuracy: 0.9943680585513983
current feature ratio: 0.19842513149602498, current accuracy: 0.9941668512753545
current feature ratio: 0.1458161299470146, current accuracy: 0.9943680585513983
current feature ratio: 0.1071554978566341, current accuracy: 0.9938649998769276
current feature ratio: 0.07874506561842957, current accuracy: 0.9923568764469111
current feature ratio: 0.05786716951795567, current accuracy: 0.9933624270147934
current feature ratio: 0.042524687505449285, current accuracy: 0.9893398198996506
current feature ratio: 0.03125, current accuracy: 0.9860208309852403
Cross validation accuracy for chosen best_feature_ratio 0.270015: 0.994368


#BEGIN <br>
Since I am using depth=10 and N=40 for the experiment, it's hard to see significant increase on the performance. However, you can see that the accuracy stops to improve when feature ratio is 0.270015

#END<br>

**Boosting**

There are different methods of boosting, but we'll focus in this problem on Adaptive Boosting (AdaBoost).
The logic of AdaBoost is to "push" each new learner to give more importance to previously misclassified data. We present
below the multiclass variant of AdaBoost [SAMME](https://web.stanford.edu/~hastie/Papers/samme.pdf). We denote $K$ the number of classes.

AdaBosst is performed by increasing the weights of misclassified simple after each iteration:
- Input: m samples $(X_i, y_i)_{i\in [m]}$, number of boosting rounds $N$
- Start with equal samples weights $W = (w_i), $ where   $w_i = \frac{1}{\texttt{n_samples}}$
- at round j:
    - Train estimator $h_j$ using current weights $W$
    - Get the predicted $(\hat{y}_i)$ on the training data using $h_j$
    - Find the weighted error rate $\epsilon_j$ using $W$: $\epsilon_j=\frac{\sum_i w_i \Delta(\hat{y}_i, y_i)}{\sum_i w_i}$
    - Choose $\alpha_j = \log \frac{1-\epsilon_j}{\epsilon_j} + \log(K-1)$
    - Update $W$ using: $w_i \leftarrow w_i \exp(\alpha_j \Delta(\hat{y_i}, y_i)) $
    - Normalize $W$ to have sum 1
- Global estimator is $H = \sum_j \alpha_j h_j$,

the $\Delta$ function equals to 1 when the two argument are different, 0 otherwise.

To understand how we implement $H$, imagine we have two classes, and we boosted for 3 rounds to get $(h_1, h_2, h_3)$,
with weights $(\alpha_1, \alpha_2, \alpha_3)$. When we want to predict the label of sample $x$, we get $(h_1(x), h_2(x), h_3(x)) = (0,1,0)$.

In this case, label $0$ gets a weight $\alpha_1+\alpha_3$, while class $1$ get weight $\alpha_2$. The predicted class is the one with
the largest weight (1 if $\alpha_2 > \alpha_1 + \alpha_3$, 0 otherwise)

### Problems 4.10- 4.11 Implementing Boosting - 6 points

- 4.10 [4 pts] Complete `fit` by building `n_estimators` of DecisionTreeClassifier, each trained on the same data but with different samples weights as detailed in the algorithm. Keep track of $(\alpha_i)$

_Hint: our weak learner (DecisionTreeClassifier) can take an argument `sample_weight` when calling the `fit` method, you'll have to use it to provide the weights $W$_

- 4.11  [2 pts] Complete `predict` method to return the predicted label using the global estimator $H$. 

_Hint: use one hot encoding of the predicted labels from the weak learners and cumulate the prediction with weights $\alpha_j$, a dictionary will also work_

Notice that if the estimator is consistent (0 error rate on the training set), AdaBoost $\alpha_j$ are no longer defined. That's why this method requires a **weak** learner like the base model we used in the previous problems.

In [56]:
class AdaBoost(object):

    def __init__(self, n_estimators, base = DecisionTreeClassifier(max_depth=4)):
        """
        :param n_estimators: number of estimators/ boosting rounds
        """
        self.n_estimators = n_estimators
        self.num_classes = None
        self.estimators = []
        self.alphas = np.zeros(n_estimators)
        self.base = base

    def fit(self, X_train, y_train):

        self.num_classes = np.unique(y_train).shape[0] # K in the algorithm
        weights = np.ones(len(X_train)) / len(X_train) # W in the algorithm
        # Workspace 4.10
        #TODO: Implement Multiclass Adaboost and keep track of the alpha_j
        #BEGIN
        self.estimators = []
        for j in range(self.n_estimators):
            h = clone(self.base)
            h.fit(X_train, y_train, sample_weight=weights)
            self.estimators.append(h)
            
            y_pred = h.predict(X_train)
            error = np.sum(weights * (y_pred != y_train)) / np.sum(weights)
            
            alpha = np.log((1 - error) / error) + np.log(self.num_classes - 1)
            self.alphas[j] = alpha
            
            weights = weights * np.exp(alpha * (y_pred != y_train))
            # Normalize W to have sum 1
            weights = weights / np.sum(weights)
        #END

    def predict(self, X_test):
        answer = 0
        # Workspace 4.11
        #TODO: get the labels returned by the global estimator defined as H
        #Hint: Use one-hot format to accumulate alphas for different classes, or a dictionary
        # The predicted label is the one that accumulates the largest sum of alphas
        #Hint: We don't need predict_proba for this one
        #BEGIN
        # y_hat is one-hot encoding of the multi-class labels
        # np.eye(k)[i] returns the one-hot encoding of size k for label i
        # np.eye(4)[2] would be [0, 0, 1, 0], np.eye(4)[0] is [1,0, 0, 0]. Clever, innit?
        
        answers = np.zeros((len(X_test), self.num_classes))
        for j, estimator in enumerate(self.estimators):
            y_hat = estimator.predict(X_test)
            y_hat_onehot = np.zeros((len(y_hat), self.num_classes), dtype=int)
            y_hat_onehot[np.arange(len(y_hat)), y_hat] = 1

            answers += self.alphas[j] * y_hat_onehot

        answer = np.argmax(answers, axis=1)
        return answer

    def score(self, X, y):
        """
        Return the mean accuracy on the given test data and labels.
        :param X: Test samples, shape (num_points, num_features)
        :param y: true labels for X, shape (num_points,)
        :return: mean accuracy
        """
        accuracy = 0
        # Workspace 1.7
        #BEGIN
        y_hat = self.predict(X)
        for i in range(len(y)):
            if y_hat[i] == y[i]:
                accuracy += 1
        
        accuracy /= len(y)
        #END
        return accuracy

### Problem 4.12 Hyperparameter tuning on Adaboost [4 points]
4.12 a [2 points] Try out different values for `max_depth` in base model and `n_estimators` to arrive at the best evaluation metrics on `ThreesAndEights` validation set after training on `ThreesAndEights` training set.

In [58]:
# Workspace 4.12a
#BEGIN
# TODO: Report accuracy and other appropriate metrics
# Make sure that the label values are >=0 if using np.eye for one-hot encoding

data.y_train = np.array([1 if y == 1 else 0 for y in data.y_train])
data.y_valid = np.array([1 if y == 1 else 0 for y in data.y_valid])

depths = np.linspace(3, 10, 5, dtype = int)
n_estimators = np.linspace(10, 40, 10, dtype = int)

k = 8

best_depth = -1
best_n_estimator = -1 
best_accuracy = 0

for depth in depths:
    for n_estimator in n_estimators:        
        classifier = AdaBoost(n_estimators=n_estimator, base=DecisionTreeClassifier(max_depth=depth))
        accuracy = k_fold_cv(classifier, k, data.X_train, data.y_train)
        print("current depth: {}, current n_estimator: {}, current accuracy: {}".format(depth, n_estimator, accuracy))

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth        
            best_n_estimator = n_estimator

print("Cross validation accuracy for chosen best depth %d, best number of estimator %d: %f" % (best_depth, best_n_estimator, best_accuracy))    
#END

current depth: 3, current n_estimator: 10, current accuracy: 0.9685213848113041
current depth: 3, current n_estimator: 13, current accuracy: 0.973047212538363
current depth: 3, current n_estimator: 16, current accuracy: 0.974354695473395
current depth: 3, current n_estimator: 20, current accuracy: 0.977070062559674
current depth: 3, current n_estimator: 23, current accuracy: 0.9797855106146756
current depth: 3, current n_estimator: 26, current accuracy: 0.9810928316122621
current depth: 3, current n_estimator: 30, current accuracy: 0.9827020040082756
current depth: 3, current n_estimator: 33, current accuracy: 0.9825007967322319
current depth: 3, current n_estimator: 36, current accuracy: 0.9829030493468738
current depth: 3, current n_estimator: 40, current accuracy: 0.9833053829302386
current depth: 4, current n_estimator: 10, current accuracy: 0.9764660358879289
current depth: 4, current n_estimator: 13, current accuracy: 0.9792825329089275
current depth: 4, current n_estimator: 16, 

In [59]:
best_n_estimator=33
best_depth=6

classifier = AdaBoost(n_estimators=best_n_estimator, base=DecisionTreeClassifier(max_depth=best_depth))
classifier.fit(data.X_train, data.y_train)
accuracy = classifier.score(data.X_valid, data.y_valid)
print("Accuracy on validation set with the best metrics: %f" % (best_accuracy))    

Accuracy on validation set with the best metrics: 0.990144


4.12 b [1 point] What are your observations with respect to `max_depth` in the base model? 

% Write-up for 4.12.b <br>
#BEGIN <br>

Since AdaBoost is designed to improve the performance of weak learners that are slightly better than 50 % accuracy, max_depth should not be large to avoid overfitting the training data. As max depth increases from a small to a large number, performance increases until a certain point and then begins to decrease. AdaBoost performs better when it has the optimal max_depth value for weak learners. 

#END<br>

4.12 c [1 point] What are your observations with respect to `n_estimators`?

% Write-up for 4.12.c <br>
#BEGIN <br>
Increasing the number of estimators improves AdaBoost performance because each new estimator receives weights from samples misclassified by previous estimators. Increasing the number of estimators beyond a certain point, however, results in overfitting and no significant improvement in performance.

#END<br>